In [19]:
import paramiko
import pandas as pd
from io import BytesIO 
import oracledb
import pandas as pd
from sqlalchemy import create_engine
import os
import logging
from datetime import datetime

# Setup Log

In [20]:
log_file = f'Logs/DailyPuller_{datetime.now().strftime('%Y-%m-%d')}.log'

In [21]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler(log_file, mode='a'),
        logging.StreamHandler()
    ]
)

# A. XPOSE Fasilitas

In [22]:
# server credentials
hostname = '10.220.42.38'
port = 22
username = os.environ['UID']
password = os.environ['UPW']

In [23]:
# create ssh client
ssh = paramiko.SSHClient()

In [24]:
# load system ssh keys
ssh.load_system_host_keys()

In [25]:
# add missing host keys automaticallly
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)

In [26]:
try:
    ssh.connect(hostname, port=port, username=username, password=password)
    sftp = ssh.open_sftp()
    remote_file_path = '/CLS/FASILITAS.DAT'
    with sftp.file(remote_file_path, 'r') as remote_file:
        file_data = remote_file.read()
    csv_file = BytesIO(file_data)
    Fasilitas = pd.read_csv(csv_file,sep='|', header=None,low_memory=False)
    FDate = Fasilitas.iloc[0,0].astype(int)
    sv_Fasilitas = f'E:/Pricing_Non_Retail/Christ/xpose/FASILITAS_{FDate}.DAT'
    Fasilitas.to_csv(sv_Fasilitas, sep="|", index=False)
    sftp.close()
    ssh.close()
    logging.info('Facility done.')
except:
    logging.warning('SFTP login failed!')     
    pass


2024-10-31 13:28:28 - INFO - Connected (version 2.0, client 6.6.2.451)


2024-10-31 13:28:29 - INFO - Authentication (password) successful!
2024-10-31 13:28:29 - INFO - [chan 0] Opened sftp connection (server version 3)
2024-10-31 14:26:10 - INFO - [chan 0] sftp session closed.
2024-10-31 14:26:10 - INFO - Facility done.


# Master Funding

In [27]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
dsn = '10.220.50.121:1661/ANALYTIC'

In [28]:
# create connection
connection = oracledb.connect(user=username, password=password, dsn=dsn)

DatabaseError: ORA-28001: the password has expired
Help: https://docs.oracle.com/error-help/db/ora-28001/

In [11]:
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [12]:
# # open and read the file as a single buffer
# fd = open('test_db.sql', 'r')
# sqlFile = fd.read()
# fd.close()

In [13]:
query = """SELECT DISTINCT BASE_DT, AGREE_ID, FLAG, ACCT_NO, ACCT_BR, REGION, AREA, BRANCH, GCIF_NO, CIF_NO,
SUB_PROD_NM, SEGMENT, GCIF_NAME, PROD_TYPE, CURR_CODE, RATE_DPK, BASE_AMOUNT, NPK_SALES, COLT
FROM PDA.MASTER_FUNDING
WHERE ((agree_id LIKE 'ST%' AND STATUS NOT IN ('03','04','05') AND system_type LIKE '8%' AND ctrl3 <>'797')
OR (agree_id like 'IM%' and status not in ('04','05','08') and system_type like '8%' and ctrl3 <>'797' ))
AND NOT SEGMENT = 'CFS-RB-5MB' 
AND BASE_DT=(SELECT MAX(BASE_DT) FROM PDA.MASTER_FUNDING)"""
# (SELECT MAX(BASE_DT) FROM PDA.MASTER_FUNDING)

In [14]:
MF = pd.read_sql(query, engine)

In [15]:
MFDate = MF.iloc[0,0]

In [16]:
sv_MF = f'E:/Pricing_Non_Retail/Ghazi/Master Funding/MASTER_FUNDING_{MFDate}.csv'

In [17]:
MF.to_csv(sv_MF, sep=",", index=False)

In [18]:
logging.info('Master funding done.')

2024-10-25 16:48:37 - INFO - Master funding done.


2024-10-25 16:54:27 - INFO - Disconnect (code 11): Login grace-time exceeded
